In [4]:
import joblib
import torch
from model import EngineRULPredictor

import pandas as pd

In [8]:
scaler = joblib.load('models/scaler.pkl')

In [10]:

input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))

model.eval()

EngineRULPredictor(
  (lstm): LSTM(18, 512, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [11]:
print("Loading model...")
input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))
model.eval()


scaler = joblib.load('models/scaler.pkl') # <--- 2. Load the scaler

col_names = ['unit_nr', 'time_cycles',
                'setting_1', 'setting_2', 'setting_3'] + [f"s_{i}" for i in range(1, 22)]
raw_df = pd.read_csv("data/test_FD001.txt", sep=r"\s+", header=None, names=col_names)
sensor_df = raw_df




Loading model...


In [ ]:
unit_nr = 1  # Example unit number
max_rul = 542  # use the same value you trained with

index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names
df = pd.read_csv("data/train_FD001.txt", sep="\s+", header=None, names=col_names)

df = df[df.unit_nr == unit_nr]

df = df.drop(columns=["unit_nr", "time_cycles"])  # Drop non-feature columns for scaling
scaled = scaler.transform(df)
df = pd.DataFrame(scaled, columns=df.columns, index=df.index)

features_to_drop = ["s_1", "s_5", "s_10", "s_16", "s_18", "s_19"]
df = df.drop(columns=features_to_drop)

input_tensor = torch.tensor(df.to_numpy(), dtype=torch.float32)
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension
input_tensor = input_tensor[:, -10:, :]  # Use last 50 time steps
print(f"Input tensor shape: {input_tensor.shape}")



# Perform prediction
with torch.no_grad():
    prediction = model(input_tensor)
    print(f"Raw model prediction: {prediction.item()}")
    scaled_prediction = prediction.item() * max_rul
    print(f"Scaled RUL prediction for unit {unit_nr}: {scaled_prediction} cycles")



Input tensor shape: torch.Size([1, 40, 18])
Raw model prediction: 0.0176466703414917
Scaled RUL prediction for unit 1: 9.564495325088501 cycles


<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_793294/474249413.py:8: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv("data/train_FD001.txt", sep="\s+", header=None, names=col_names)


tensor([[[2.0944e-04, 8.3076e-04, 1.0000e+00, 9.8294e-01, 9.4741e-01,
          9.4623e-01, 1.0000e+00, 9.5530e-01, 9.9909e-01, 8.3838e-01,
          9.5997e-01, 9.5738e-01, 9.9377e-01, 6.0727e-01, 1.1949e-01,
          9.5918e-01, 9.5526e-01, 9.5428e-01],
         [2.1658e-04, 1.0681e-03, 1.0000e+00, 9.9051e-01, 9.6935e-01,
          9.7853e-01, 1.0000e+00, 9.5442e-01, 9.9916e-01, 8.4391e-01,
          9.6317e-01, 9.5669e-01, 9.9391e-01, 6.0238e-01, 1.2266e-01,
          9.4898e-01, 9.5089e-01, 9.4783e-01],
         [2.2610e-04, 8.3076e-04, 1.0000e+00, 9.8805e-01, 9.6406e-01,
          9.5909e-01, 1.0000e+00, 9.5589e-01, 9.9926e-01, 8.4058e-01,
          9.6958e-01, 9.5682e-01, 9.9388e-01, 5.9809e-01, 1.1754e-01,
          9.4898e-01, 9.5358e-01, 9.5092e-01],
         [2.7132e-04, 3.5604e-04, 1.0000e+00, 9.8541e-01, 9.4188e-01,
          9.6364e-01, 1.0000e+00, 9.5803e-01, 9.9909e-01, 8.4666e-01,
          9.6717e-01, 9.5745e-01, 9.9383e-01, 6.1988e-01, 1.2588e-01,
          9.6939e-0